<img src="https://habrastorage.org/webt/3x/l-/yv/3xl-yvgopmrwrjaz7hvkouaplye.png" />

_Автор: Лопухова Арина (Slack: @erynn)_


## Содержание <a name="cont"></a>
1. [Описание задачи и данных](#intro)
2. [Создание признаков I](#feat_1)
3. [Первичный и визуальный анализ. Найденные зависимости](#analysis)
4. [Создание признаков II](#feat_2)
5. [Выбор моделей и метрики](#model)
6. [Предобработка данных, обучение, оценка результатов на LB](#learning)
    
    6.1 [KNeighborsRegressor](#knr)
    
    6.2 [KNeighborsRegressor на расширенной выборке](#knr)
    
    6.3 [RandomForestRegressor](#rfr)
    
    6.4 [XGBRegressor](#xgb)

7. [Почти выводы](#alm_conc)
8. [Выводы](#conc)

## Описание задачи и данных <a name="intro"></a>

[К началу](#cont)

Подробное описание задачи и данных можно посмотреть [на странице соревнования.](https://datasouls.com/c/mts-geohack/description) Задача в том, чтобы по среднему числу звонков в экстренные службы с участков примерно 500x500 метров в западной части Москвы и Московской области спрогнозировать число звонков с таких участков в восточной части Москвы и Московской области по каждому дню недели. Данные 2017 года.  

Качество оценивается через [коэффициент ранговой корреляции Кендалла](http://www.machinelearning.ru/wiki/index.php?title=%D0%9A%D0%BE%D1%8D%D1%84%D1%84%D0%B8%D1%86%D0%B8%D0%B5%D0%BD%D1%82_%D0%BA%D0%BE%D1%80%D1%80%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D0%B8_%D0%9A%D0%B5%D0%BD%D0%B4%D0%B5%D0%BB%D0%BB%D0%B0) независимо по каждому дню недели. 

Задача интересна тем, что потенциально может выявить проблемные районы, которым нужны изменения в инфраструктуре или больше внимания экстренных служб. Она также полезна как анализ нагрузки на сотового оператора (все-таки данные предоставляет МТС). Плюс задача хороша в учебно-исследовательском плане, потому что предполагает работу с геоданными, к тому же из внешних источников вроде [OpenStreetMap](https://www.openstreetmap.org/). Готовых признаков вообще по сути нет, и все нужно делать вручную. 

Данные (train + test в одном файле и все остальные дополнительные данные) можно скачать [отсюда](https://drive.google.com/file/d/10F166_vlWkP2-z526nTaiWG4efMyIf8a/view?usp=sharing).

Значения переменных:
* __lat_bl, lon_bl__ — координаты нижнего левого угла участка
* __lat_tr, lon_tr__ — координаты верхнего правого угла участка
* __lat_c, lon_c__ — координаты центра участка
* __calls_daily__ — среднее число вызовов по всем дням
* __calls_workday__ — среднее число вызовов по рабочим дням
* __calls_weekend__ — среднее число вызовов по выходным дням
* __calls_wd{D}__ — среднее число вызовов по дню недели D (0 — понедельник, 6 — воскресенье)
* __is_test__ — индикатор того, что объект тестовый (у тестовых объектов значения числа звонков не показаны)
* __is_target__ — индикатор целевого участка (указан только для обучающей выборки)

Про целевой участок: качество предсказания оценивается не по всей тестовой части, а только по тем участкам, которые входят в целевые. Участки из тестовой выборки отбираются в целевые по тому же принципу, что и участки обучающей (по какому — участникам неизвестно).

Целевой признак — __calls_wd{D}__.

__Примечание к работе:__ в [репозитории](https://github.com/datasouls/mts-geohack/blob/master/Geohack112_StarterKit.ipynb) соревнования есть базовая тетрадка Starter Kit, части которой здесь будут использоваться. Оттуда взяты идеи про расстояние до Кремля и близость к каким-то объектам/число объектов в радиусе (часть про конструирование признаков). Взятый из Kit код будет помечен как #SK.

__Установка osmread для работы с OpenStreetMap и импорт библиотек__

In [ ]:
# !pip install osmread

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from scipy.stats import kendalltau
import osmread

from tqdm import tqdm_notebook
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import os

In [ ]:
%matplotlib inline
#размер графиков
mpl.rcParams['figure.figsize'] = (8, 7)
#стиль и размер шрифта
sns.set(style='whitegrid', font_scale=1.5)

In [ ]:
PATH_TO_DATA = './data/'  #путь к данным 

distance из cython_dist — реализованная на cython функция для подсчета расстояния в км между точками в географических координатах

In [ ]:
from cython_dist import distance

__Считаем файл с участками__

In [ ]:
zones_df = pd.read_csv('data/zones.csv', index_col='zone_id')
zones_df.head()

__Считаем из tagged_nodes.pickle объекты из OSM__, которые попали в интересующий нас регион и имеют хотя бы один тег. Подробно о том, как шла работа с картой и создавался tagged_nodes.pickle, можно посмотреть в [Starter Kit](https://github.com/datasouls/mts-geohack/blob/master/Geohack112_StarterKit.ipynb), раздел "Работа с данными OpenStreetMap".  

Если вкратце про объекты карты OSM, то их три типа (описание из того же Starter Kit): 
* __Node__  — точки на карте
* __Way__  — дороги, площади, задающиеся набором точек
* __Relation__  — связи между элементами

В tagged_nodes.pickle только объекты типа Node, которые попали в большой прямоугольник от Наро-Фоминска на юго-западе от Москвы до Красноармейска на северо-востоке от Москвы.

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'tagged_nodes.pickle'), 'rb') as f:
    tagged_nodes = pickle.load(f)

__Пример объекта__

In [ ]:
tagged_nodes[17]

## Создание признаков I <a name="feat_1"></a>

[К началу](#cont)

С этого все начнется, потому что использовать готовые данные в виде чистых координат не имеет смысла. Уже после накидывания признаков будет проведен их анализ.

__Расстояние до Кремля__

Мотивация: районы ближе к центру воспринимаются как более благополучные по сравнению с окраинами, что должно влиять на количество звонков в полицию.

__Признаки вида: 1) число объектов какого-то типа в определенном радиусе от центра участка; 2) расстояние до k ближайших объектов какого-то типа; 3) расстояние от центра участка до ближайшего объекта определенного типа__

Под объектами имеются в виду объекты OpenStreetMap.

Мотивация:
* __Магазины__: 1) плотная застройка магазинами позволяет отделить более старые обжитые районы от новых микрорайонов с новостройками ближе к окраинам; 2) магазины — открытые многолюдные места, которые обеспечивают поток людей на ближайших улицах, что должно уменьшать вероятность преступлений на улице 
* __Рестораны, кафе + кино, театр + каток, площадка, парк__: аналогичный поток людей и признак благоустроенности
* __Общественный транспорт__: вот тут сложнее, потому что с одной стороны это близость людей, с другой — особенно плотное скопление людей, давка и потенциальная опасность здоровью плюс возможность ДТП, поэтому пока неясно, в какую сторону этот признак может влиять на целевой

Признаки могут считаться довольно долго из-за пользовательской метрики расстояния, которая передается в NearestNeighbours, поэтому они уже заранее посчитаны и лежат в dist_features.pkl. Про расшифровку тегов объектов OSM можно почитать
[здесь.](https://wiki.openstreetmap.org/wiki/Map_Features)

Примечание про метрику расстояния: в исходном SK расстояние между точками в градусах считалось как евклидово, в моей версии оно считается именно как расстояние между точками на эллипсоиде ради интерпретируемости и возможного улучшения качества.

In [ ]:
dist_features = pd.DataFrame(index=zones_df.index)

#признак вида "расстояние до Кремля"
kremlin_lat, kremlin_lon = 55.753722, 37.620657
dist_features['kremlin_dist'] = zones_df.apply(lambda row: \
                                             distance(np.array([row.lat_c, row.lon_c]), np.array([kremlin_lat, kremlin_lon])), \
                                             axis=1)

In [ ]:
#SK
#набор фильтров точек, по которым будет считаться статистика

POINT_FEATURE_FILTERS = [
    ('tagged', lambda node: len(node.tags) > 0),
    ('railway', lambda node: node.tags.get('railway') == 'station'),
    ('public_transport', lambda node: 'public_transport' in node.tags),
    ('entertainment', lambda node: node.tags.get('amenity') in ['cinema', 'theatre']),
    ('food', lambda node: node.tags.get('amenity') in ['restaurant', 'cafe', 'fast_food']),
    ('leisure', lambda node: node.tags.get('leisure') not in [None, 'amusement_arcade', 'adult_gaming_centre']),
    ('shop', lambda node: node.tags.get('shop') not in [None, 'lottery'])
]

# центры квадратов в виде матрицы
X_zone_centers = zones_df[['lat_c', 'lon_c']].as_matrix()

for prefix, point_filter in tqdm_notebook(POINT_FEATURE_FILTERS):

    # берем подмножество точек в соответствии с фильтром
    coords = np.array([
        [node.lat, node.lon]
        for node in tagged_nodes
        if point_filter(node)
    ])

    # строим структуру данных для быстрого поиска точек
    neighbors = NearestNeighbors(metric=distance).fit(coords)
    
    # признак вида "количество точек в радиусе R от центра квадрата"
    for radius in tqdm_notebook([0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25]):
        dists, inds = neighbors.radius_neighbors(X=X_zone_centers, radius=radius)
        dist_features['{}_points_in_{}'.format(prefix, radius)] = np.array([len(x) for x in inds])

    # признак вида "расстояние до ближайших K точек"
    for n_neighbors in tqdm_notebook([3, 5, 10]):
        dists, inds = neighbors.kneighbors(X=X_zone_centers, n_neighbors=n_neighbors)
        dist_features['{}_mean_dist_k_{}'.format(prefix, n_neighbors)] = dists.mean(axis=1)
        dist_features['{}_max_dist_k_{}'.format(prefix, n_neighbors)] = dists.max(axis=1)
        dist_features['{}_std_dist_k_{}'.format(prefix, n_neighbors)] = dists.std(axis=1)

    # признак вида "расстояние до ближайшей точки"
    dist_features['{}_min'.format(prefix)] = dists.min(axis=1)

__Считаем признаки из файла__

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'features/dist_features.pkl'), 'rb') as f:
    dist_features = pickle.load(f)    

In [ ]:
dist_features.head()

__Переходы через проезжую часть/железную дорогу__:

* число пешеходных переходов
* число и доля нерегулируемых пешеходных переходов

Мотивация: переход через дорогу, особенно в нерегулируемых местах, связан с риском аварии, что в свою очередь связано со звонками в скорую. Проверить, зайдет ли доля объектов в качестве признака, в голову пришло после тренировки по машинному обучению в Яндексе, и там это просто сработало в одном соревновании.

In [ ]:
def in_zone(zone_coord, obj_coord):
    '''
    in: iterable zone_coord (lat_bl, lon_bl, lat_tr, lon_tr) 
        iterable obj_coord (lat, lon)
    out: True, если объект внутри зоны, иначе False
    '''
    lat_bl, lon_bl, lat_tr, lon_tr = zone_coord
    lat, lon = obj_coord
    
    return (lat_bl <= lat <= lat_tr) and (lon_bl <= lon <= lon_tr)

In [ ]:
def obj_count(zone_coord, nodes):
    '''
    in: iterable zone_coord (lat_bl, lon_bl, lat_tr, lon_tr)
        list  nodes (список координат объектов одного интересующего типа)
    out: число объектов указанного типа внутри участка
    '''
    count = 0
    for node in nodes:
        if in_zone(zone_coord, node):
            count += 1
    return count        

In [ ]:
def select_type(types, tagged_nodes=tagged_nodes):
    '''
    in: str types (строка вида 'key1=value1,key2=value2, ...')  с интересующими парами ключ-значение
        либо только ключом (тогда проверяется, что ключ есть среди ключей объекта OSM)
        list tagged_nodes (объекты OSM)
    out: list (список списков, где во внутренних списках — координаты объектов OSM, соответствующих парам из types)
    '''    
    outer, inner = [], []
    for pair in types.split(sep=','):
        try:
            key, value = pair.split(sep='=')
        except Exception:
            key = pair
            for node in tagged_nodes:
                if key in node.tags:
                    inner.append((node.lat, node.lon)) 
        else:            
            for node in tagged_nodes:
                if node.tags.get(key) == value:
                    inner.append((node.lat, node.lon))

        outer.append(inner.copy())
        inner.clear()
        
    return outer    

In [ ]:
crossing_nodes, crossing_uncontr_nodes = select_type('crossing,crossing=uncontrolled') 

In [ ]:
%%time
n_crossing_nodes = zones_df.apply(lambda row: obj_count((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
                                                        crossing_nodes), axis=1)

In [ ]:
%%time
n_crossing_uncontr_nodes = zones_df.apply(lambda row: obj_count((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
                                                      crossing_uncontr_nodes), axis=1)

In [ ]:
prt_crossing_uncontr_nodes = (n_crossing_uncontr_nodes / n_crossing_nodes)

In [ ]:
crossing_features = pd.DataFrame(data={'n_crossing': n_crossing_nodes,
                                       'n_crossing_uncontr': n_crossing_uncontr_nodes,
                                       'prt_crossing': prt_crossing_uncontr_nodes})

__Можно считать из файла__

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'features/crossing_features.pkl'), 'rb') as f:
    crossing_features = pickle.load(f)

__Признаки, связанные с жилыми домами:__

* число жилых домов в участке
* число и доля жилых домов, введенных в эксплуатацию раньше 1970
* число и доля жилых домов по индивидуальному проекту
* число и доля жилых домов, к которым не проведен газ
* число и доля жилых домов с автоматической пожарной системой 

Мотивация: 

* в старых домах обычно живет много пожилых людей, которым чаще может потребоваться скорая помощь (вообще тут можно засомневаться и вспомнить про реновацию в Москве, но в 2017 году массового переселения в новые дома еще не было)
* индивидуальные проекты как характеристика района и, предположительно, благоустроенности 
* доступ к газу связан с риском ЧС
* автоматические системы тушения, предположительно, могут влиять на частоту вызовов в пожарные службы

Признаки получаются на основе данных из house_moscow_info.csv и house_nonmoscow_info.csv, которые были вытащены [отсюда](https://www.reformagkh.ru/opendata) (реестр домов по городу Москве и Московской области).

__Считываем данные__

In [ ]:
house_moscow_info = pd.read_csv(os.path.join(PATH_TO_DATA, 'house_moscow_info.csv'),
                         sep=';', 
                         usecols=['id', 'address', 'exploitation_start_year', 
                                  'project_type', 'gas_type', 'firefighting_type'])

In [ ]:
house_moscow_info.head()

In [ ]:
house_nonmoscow_info = pd.read_csv(os.path.join(PATH_TO_DATA, 'house_nonmoscow_info.csv'),
                         sep=';',
                         usecols=['id', 'formalname_city', 'address', 'exploitation_start_year', 
                                  'project_type', 'gas_type', 'firefighting_type'])

Из Московской области возьмем только топ-50 крупнейших городов: если посмотреть на целевые зоны в обучающей выборке и на карту, где эти зоны отмечены, то можно предположить, что целевыми считаются крупнейшие поселения. 

__Считаем список крупнейших городов МО__

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'top_c.txt')) as f:
    top_c = f.readlines()
    
    
top_c = [s.strip('\n').strip('\ufeff') for s in top_c] 

In [ ]:
house_nonmoscow_info = house_nonmoscow_info[house_nonmoscow_info.formalname_city.isin(top_c)]

In [ ]:
house_nonmoscow_info.head()

__Приведем записи в текстовых колонках к одному формату__

Например, значения вроде "индивидуальный", "инд." и подобные в колонке project_type все переделаем в "инд", чтобы потом правильно отбирать дома по условию "индивидуальный проект/не индивидуальный проект".

In [ ]:
ind_proj_names_moscow = [s for s in list(house_moscow_info.project_type.value_counts().index) if 'инд' in s]
ind_proj_names_moscow

In [ ]:
house_moscow_info.loc[house_moscow_info.project_type.isin(ind_proj_names_moscow), 'project_type'] = 'инд'

In [ ]:
ind_proj_names_nonmoscow = [s for s in list(house_nonmoscow_info.project_type.value_counts().index) if 'инд' in s]
ind_proj_names_nonmoscow

In [ ]:
house_nonmoscow_info.loc[house_nonmoscow_info.project_type.isin(ind_proj_names_nonmoscow), 'project_type'] = 'инд'

У типов системы газоснабжения и противопожарной системы все в одном формате.

In [ ]:
house_moscow_info['gas_type'].value_counts()

In [ ]:
house_moscow_info['firefighting_type'].value_counts()

In [ ]:
house_nonmoscow_info['gas_type'].value_counts()

In [ ]:
house_nonmoscow_info['firefighting_type'].value_counts()

__Сразу несколько проблем__

У OSM проблемы с жилыми домами (дома не отмечают как жилые либо вообще нет тега "здание"), поэтому использовалась библиотека __geocoder__, которая связывалась с API Яндекс карт и получала координаты адресов из house_moscow_info и house_nonmoscow_info, чтобы на их основе создать новые признаки (координат в чистом виде среди признаков нет). 

Проблема в том, что выкладывать координаты в открытый доступ в виде файла нарушает правила пользования сервисом, поэтому дальше будет закомментированный код-пример того, как на основе координат создавались признаки. При этом самих координат в файлах проекта нет и вручную выполнить код в этих ячейках не получится. Готовый датафрейм с этими признаками можно считать из файла чуть ниже. 

Это все довольно чудовищно выглядит, но пока это лучшее решение проблемы, до которого удалось дойти.

В house_moscow_lat_lon и house_nonmoscow_lat_lon лежали координаты адресов домов из house_moscow_info и house_nonmoscow_info.

In [ ]:
# house_moscow_info['lat_lon'] = house_moscow_lat_lon
# house_nonmoscow_info['lat_lon'] = house_nonmoscow_lat_lon

In [ ]:
# full_house_info = pd.concat([house_moscow_info, house_nonmoscow_info.drop('formalname_city', axis=1)], 
#                             ignore_index=True)

__Генерируем признаки__

Попробуем только для целевых и тестовых квадратов. Получившиеся признаки можно сразу считать из house_features.pkl.

Среди координат в full_house_info попадаются -1: это когда Яндекс-карты не распознавали адрес. Такие адреса выкинем.

In [ ]:
# mask = full_house_info['lat_lon'].apply(lambda lst: type(lst) == int)
# corrupted_coord = full_house_info['lat_lon'][mask]
# full_house_info.drop(corrupted_coord.index, inplace=True)

In [ ]:
# %%time
# n_resident_house = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)] \
#                   .apply(lambda row: obj_count((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
#                                                 full_house_info['lat_lon']), axis=1)

In [ ]:
# %%time
# n_old = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)] \
#         .apply(lambda row: obj_count((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
#                                       full_house_info[full_house_info.exploitation_start_year <= 1970]['lat_lon']), axis=1)
# prt_old = n_old / n_resident_house

In [ ]:
# %%time
# n_ind_proj = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)] \
#             .apply(lambda row: obj_count((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
#                                          full_house_info[full_house_info.project_type == 'инд']['lat_lon']), axis=1)
# prt_ind_proj = n_ind_proj / n_resident_house    

In [ ]:
# %%time
# n_without_gas = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)] \
#                 .apply(lambda row: obj_count((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
#                                              full_house_info[full_house_info.gas_type == 'Отсутствует']['lat_lon']), axis=1)
# prt_without_gas = n_without_gas / n_resident_house    

In [ ]:
# %%time
# n_auto_fire = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)] \
#              .apply(lambda row: obj_count((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
#                                           full_house_info[full_house_info.firefighting_type == 'Автоматическая']['lat_lon']), axis=1)
# prt_auto_fire = n_auto_fire / n_resident_house    

In [ ]:
# full_house_features = pd.DataFrame({'n_resident': n_resident_house, 'n_old': n_old, 'n_ind_proj': n_ind_proj,
#                                     'n_without_gas': n_without_gas, 'n_auto_fire': n_auto_fire,
#                                     'prt_old': prt_old, 'prt_ind_proj': prt_ind_proj,
#                                     'prt_without_gas': prt_without_gas, 'prt_auto_fire': prt_auto_fire})

__Тоже можно считать из файла__

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'features/house_features.pkl'), 'rb') as f:
    full_house_features = pickle.load(f)

__Число базовых станций МТС в участке__

Координаты БС взяты [отсюда](http://bsmaps.ru/viewtopic.php?f=5&t=358) (оставлены только координаты нужного нам региона).

Мотивация: предположение о том, что больше вышек там, где плотность звонков выше либо область значима.

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'stat_coords.pkl'), 'rb') as f:
    stat_coords = pickle.load(f)

In [ ]:
%%time
n_base_stations = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)] \
                  .apply(lambda row: obj_count((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
                                                stat_coords), axis=1)

__И это можно считать из файла__

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'features/base_stat.pkl'), 'rb') as f:
    n_base_stations = pickle.load(f)

__Признак: есть ли в участке объект определенного типа__

Мотивация: просто попробовать, потому что объекты значимые.

In [ ]:
def obj_present(zone_coord, nodes):
    '''
    in: iterable zone_coord (lat_bl, lon_bl, lat_tr, lon_tr)
        list  nodes (список координат объектов одного интересующего типа)
    out: 1/0 (объект есть в участке/нет)
    '''
    for node in nodes:
        if in_zone(zone_coord, node):
            return 1
    return 0

In [ ]:
clinic_nodes, police_nodes, school_nodes = select_type('amenity=clinic,amenity=police,amenity=school') 

In [ ]:
%%time
has_clinic = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)] \
            .apply(lambda row: obj_present((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
                                            clinic_nodes), axis=1)

In [ ]:
%%time
has_police = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)] \
            .apply(lambda row: obj_present((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
                                            police_nodes), axis=1)

In [ ]:
%%time
has_school = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)] \
            .apply(lambda row: obj_present((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
                                            school_nodes), axis=1)

In [ ]:
has_obj_features = pd.DataFrame(data={'has_clinic': has_clinic,
                                      'has_police': has_police,
                                      'has_school': has_school})

## Первичный + визуальный анализ данных

## Найденные зависимости <a name="analysis"></a>

[К началу](#cont)

__Создадим сводный DataFrame признаков и выделим train, test и звонки в train__

In [ ]:
idx_full = zones_df[(zones_df.is_test == 1) | (zones_df.is_target == 1)].index
full_features = pd.concat([dist_features.loc[idx_full],
                           full_house_features,
                           crossing_features.loc[idx_full],
                           n_base_stations.rename('n_base_stations'),
                           has_obj_features],
                           axis=1)

In [ ]:
idx_train, idx_test = zones_df[zones_df.is_target == 1].index, zones_df[zones_df.is_test == 1].index
train_features, calls_train = full_features.loc[idx_train], zones_df.loc[idx_train, :].iloc[:, 8:] 
test_features = full_features.loc[idx_test]

In [ ]:
train_features.shape, test_features.shape

__Посмотрим на значения NaN и незаполненные значения__

Все признаки создавались вручную, так что можно сразу сказать, где есть проблемы. Незаполненных значений нет. NaN есть в тех колонках, которые отвечают за долю каких-то объектов. Появляются, очевидно, когда считается доля от нуля. Например:

In [ ]:
full_features['prt_old'].value_counts(dropna=False).head()

__Диапазоны значений оценим по каждому типу признаков отдельно__

__По признакам расстояния__

In [ ]:
dist_features.loc[idx_full].describe()

In [ ]:
dist_features.loc[idx_full].describe().iloc[:, 1:17]

Максимальное значение tagged_points напрягает очень большой разницей по сравнению с верхним квартилем. Изобразим точки, плотно окруженные объектами OSM, с помощью __folium__.

In [ ]:
# !pip install folium

In [ ]:
import folium

In [ ]:
quant_99 = dist_features.loc[idx_full, 'tagged_points_in_0.25'].quantile(0.99)
dense_mask = dist_features.loc[idx_full, 'tagged_points_in_0.25'] > quant_99

In [ ]:
dense_zones = zones_df.loc[idx_full][dense_mask]

In [ ]:
#начальным положением зададим Красную площадь
fmap = folium.Map([55.753722, 37.620657])

for _, row in dense_zones.iterrows():
    folium.CircleMarker([row.lat_c, row.lon_c], radius=4).add_to(fmap)
    
fmap    

Основная масса точек лежит близко к центру Москвы, что, как мне кажется оправдывает такую плотность объектов в участке.

__Посмотрим на другие признаки из dist_features__

In [ ]:
dist_features.loc[idx_full].describe().iloc[:, 17:33]

In [ ]:
dist_features.loc[idx_full].describe().iloc[:, 33:49]

In [ ]:
dist_features.loc[idx_full].describe().iloc[:, 49:65]

In [ ]:
dist_features.loc[idx_full].describe().iloc[:, 65:81]

In [ ]:
dist_features.loc[idx_full].describe().iloc[:, 81:97]

In [ ]:
dist_features.loc[idx_full].describe().iloc[:, 97:113]

В целом опять внимание привлекает разница между верхним квартилем и максимальным значением признаков. __Еще раз посмотрим на карту__, на этот раз отметим центры участков, которые плотно окружены магазинами.

In [ ]:
quant_99 = dist_features.loc[idx_full, 'shop_points_in_0.25'].quantile(0.99)
dense_mask = dist_features.loc[idx_full, 'shop_points_in_0.25'] > quant_99

In [ ]:
dense_zones = zones_df.loc[idx_full][dense_mask]

In [ ]:
#начальным положением зададим Красную площадь
fmap = folium.Map([55.753722, 37.620657])

for _, row in dense_zones.iterrows():
    folium.CircleMarker([row.lat_c, row.lon_c], radius=4, color='#DC143C').add_to(fmap)
    
fmap    

Если вручную потыкать по карте, то можно увидеть, что какие-то отмеченные точки просто находятся рядом с торговыми центрами, отсюда и такое число магазинов и мест с тегом "food".

__По признакам, связанным с жилыми домами__

In [ ]:
full_house_features.describe()

__Посмотрим на жилые дома на карте__

In [ ]:
quant_99 = full_house_features.loc[idx_full, 'n_resident'].quantile(0.99)
dense_mask = full_house_features.loc[idx_full, 'n_resident'] > quant_99

In [ ]:
dense_zones = zones_df.loc[idx_full][dense_mask]

In [ ]:
#начальным положением зададим Красную площадь
fmap = folium.Map([55.753722, 37.620657])

for _, row in dense_zones.iterrows():
    folium.CircleMarker([row.lat_c, row.lon_c], radius=4, color='#DC143C').add_to(fmap)
    
fmap    

Большое количество жилых домов особенно не смущает: на карте можно видеть места со спальными районами вроде Балашихи, Химок, Мытищ. Больше 70% данных — это участки без жилых домов вообще, и вот это настораживает больше. 

__По числу переходов и БС__

Переходы:

In [ ]:
crossing_features.describe()

Здесь аналогичный разрыв между максимумом и верхним квартилем.

БС:

In [ ]:
n_base_stations.describe()

In [ ]:
quant_99 = n_base_stations[idx_full].quantile(0.99)
dense_mask = n_base_stations[idx_full] > quant_99

In [ ]:
dense_zones = zones_df.loc[idx_full][dense_mask]

In [ ]:
#начальным положением зададим Красную площадь
fmap = folium.Map([55.753722, 37.620657])

for _, row in dense_zones.iterrows():
    folium.CircleMarker([row.lat_c, row.lon_c], radius=4, color='#DC143C').add_to(fmap)
    
fmap    

Снова скопление точек в центре Москвы.

__Посмотрим на целевой признак__

Из-за особенностей метрики качества (важен порядок, а не точные значения), можно, вообще говоря, делать одинаковые предсказания на каждый день недели (так делали в Starter Kit) основываясь на среднем числе звонков в день.

In [ ]:
calls_train.describe()

__Посмотрим на распределение звонков__ по дням недели, по будням/выходным и просто в среднем по дням.

__По дням недели__

In [ ]:
fig, ax_list = plt.subplots(3, 3, figsize=(15, 10))

n = 0
for i in range(3):
    for j in range(3):
        sns.kdeplot(calls_train['calls_wd{}'.format(n)], ax=ax_list[i][j]);
        n += 1
        if n > 6:
            break

Уже по графику видно, что распределение ассимметрично. Можно формально проверить на нормальность и ожидаемо отвергнуть гипотезу о нормальности. 

In [ ]:
from scipy.stats import normaltest

alpha = 0.05
_, pvalue = normaltest(calls_train['calls_wd0'])
pvalue > alpha

__По будням/выходным__

In [ ]:
sns.kdeplot(calls_train['calls_workday'])
sns.kdeplot(calls_train['calls_weekend'], color='red');

__В среднем по всем дням__

In [ ]:
sns.kdeplot(calls_train['calls_daily']);

Главный вывод из графиков распределения: целевые признаки не распределены нормально, значения сильно группируются вблизи нуля. Возможно, стоит преобразовать целевой признак, чтобы избавиться от сильной ассимметричности.

__Посмотрим на boxplot звонков по дням недели__

In [ ]:
sns.boxplot(data=[calls_train['calls_wd{}'.format(i)] for i in range(7)]);

Здесь хорошо видны выбросы: по всем дням недели есть зоны, у которых число звонков заходит сильно выше верхнего квартиля. 

__Посмотрим поближе__

In [ ]:
quant_99 = [calls_train['calls_wd{}'.format(i)].quantile(0.99) for i in range(7)]
mx = [calls_train['calls_wd{}'.format(i)].max() for i in range(7)]
plt.scatter(x=range(7), y=quant_99, c='red')
plt.scatter(x=range(7), y=mx, c='black');

__И на boxplot звонков по будням/выходным__

In [ ]:
sns.boxplot(data=[calls_train.calls_workday, calls_train.calls_weekend]);

Из этих графиков можно сделать вывод о том, что есть участки, у которых среднее число вызовов нехарактерно большое. Причем, скорее всего, дело не в самих участках, а в каком-то ЧП, которое там произошло. Пожар, например, из-за которого могло резко увеличиться число звонков. Думаю, что при использовании линейной регрессии такие участки нужно исключить из выборки из-за нетипичности.

Еще можно посмотреть на эти участки на карте. Они не локализуются, то есть если это все-таки ЧП, то разные ЧП. 

In [ ]:
quant_99 = calls_train['calls_daily'].quantile(0.99)
mask = calls_train['calls_daily'] > quant_99

In [ ]:
many_calls_zones = zones_df.loc[idx_train][mask]

In [ ]:
#начальным положением зададим Красную площадь
fmap = folium.Map([55.753722, 37.620657])

for _, row in many_calls_zones.iterrows():
    folium.CircleMarker([row.lat_c, row.lon_c], radius=4, color='#DC143C').add_to(fmap)
    
fmap    

__Попробуем посмореть, как коррелируют между собой признаки и как они влияют на целевой__

__Для dist_features__

tagged_points будут коррелировать со всеми, так что посмотрим на корреляцию признаков вида "число объектов в заданном радиусе" без tagged_points:

In [ ]:
p_list = ['railway', 'public_transport', 'entertainment', 'food', 'leisure', 'shop']

cmap = sns.cm.rocket_r
sns.heatmap(dist_features.loc[idx_full, ['{}_points_in_0.1'.format(p) for p in p_list]]
            .corr('kendall'), cmap=cmap);

In [ ]:
sns.heatmap(dist_features.loc[idx_full, ['{}_points_in_0.25'.format(p) for p in p_list]]
            .corr(), cmap=cmap);

Тут никаких полезных инсайтов, только бытовые наблюдения: коррелирует транспорт (в городе вокзалы часто рядом с метро и автобусными остановками), самые сильные корреляции — c магазинами (если в участке много магазинов, то там, скорее всего, есть большой торговый центр, в котором есть фуд-корты и которые стоят недалеко от остановок/метро). Возьмем побольше признаков.

__Для dist_features, house_features и base_stations__

In [ ]:
sns.heatmap(full_features.loc[idx_full, ['{}_points_in_0.25'.format(p) for p in p_list] +
                                        ['n_resident', 'n_old', 'n_ind_proj', 'n_auto_fire', 'n_without_gas'] +
                                        ['n_base_stations']]
            .corr(), cmap=cmap);

Число жилых домов сильно коррелирует с числом домов старше 1970 года и домов без газоснабжения. Из интересного вижу разве что корреляцию между числом базовых станций и магазинами и местами, где можно поесть: это, возможно, опять можно объяснить торговыми центрами.

__Посмотрим на влияние признаков на целевой__

Поскольку распределение звонков по дням недели слабо отличается, посмотрим на звонки не по дням недели, а по будням/выходным и в среднем по всем дням.

In [ ]:
sns.heatmap(pd.concat([calls_train[['calls_daily', 'calls_workday', 'calls_weekend']],
                       train_features.loc[:, ['{}_points_in_0.25'.format(p) for p in p_list] +
                                             ['{}_mean_dist_k_5'.format(p) for p in p_list]] 
                      ],
                       axis=1)
                       .corr(), cmap=cmap);

Здесь звонки сильнее всего коррелируют с объектами, относящимися к общественному транспорту, с магазинами и кафе/ресторанами, причем положительно, хотя было предположение, что в случае с магазинами должно быть наоборот. Возможное объяснение: в торговых центрах могут быть пожары, при которых звонят сразу много людей (об этом выше), а рядом с остановками — аварии.

При этом со средним расстоянием до разных объектов звонки коррелируют слабо, то есть либо связь сильно нелинейная, либо связь в принципе слабая. Ниже посмотрим на графики; возможно, эти признаки можно будет вообще убрать.

In [ ]:
sns.heatmap(pd.concat([calls_train[['calls_daily', 'calls_workday', 'calls_weekend']],
                       train_features.loc[:, ['{}_min'.format(p) for p in p_list] +
                                             ['n_crossing'] +
                                             ['n_crossing_uncontr'] +
                                             ['kremlin_dist']]
                      ],
                       axis=1)
                       .corr(), cmap=cmap);

Сильнее всего звонки коррелируют с числом переходов. С минимальным расстоянием до объектов, включая Кремль, по силе корреляция меньше.

In [ ]:
sns.heatmap(pd.concat([calls_train[['calls_daily', 'calls_workday', 'calls_weekend']],
                       train_features.loc[:, ['n_resident', 'n_old', 'n_ind_proj',
                                              'n_auto_fire', 'n_without_gas', 'n_base_stations']]
                      ],
                       axis=1)
                       .corr(), cmap=cmap);

Сильнее всего звонки коррелируют с числом БС. А вот с домами довольно слабо.

In [ ]:
sns.heatmap(pd.concat([calls_train[['calls_daily', 'calls_workday', 'calls_weekend']],
                       train_features.loc[:, ['has_clinic', 'has_police', 'has_school']]
                      ],
                      axis=1)
                      .corr(method='spearman'), cmap=cmap);

Есть слабая корреляция звонков с числом больниц.

__Посмотрим на графики зависимости__ среднего числа звонков в день от признаков, с которыми имеется наиболее сильная корреляция.

__Влияние числа объектов в радиусе 250 м__:

In [ ]:
fig, ax_list = plt.subplots(2, 3, figsize=(16, 10))

ax_num = 0
for p in p_list:
    sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
                x='{}_points_in_0.25'.format(p), 
                y='calls_daily',
                ax=ax_list.reshape(-1, 1)[ax_num][0]);
    ax_num += 1

Что-то похожее на тренд, если не учитывать явные выбросы, видится разве что для shop_points и food_points. У остальных признаков виден сильный разброс относительно регрессионной прямой, которую строит Seaborn.

__Влияние числа переходов, БС и жилых домов__:

In [ ]:
fig, ax_list = plt.subplots(2, 2, figsize=(16, 8))

sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
            x='n_crossing', 
            y='calls_daily',
            ax=ax_list[0][0]);
sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
            x='n_crossing_uncontr', 
            y='calls_daily',
            ax=ax_list[0][1]);
sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
            x='n_base_stations', 
            y='calls_daily',
            ax=ax_list[1][0]);
sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
            x='n_resident', 
            y='calls_daily',
            ax=ax_list[1][1]);

У n_crossing и n_crossing_uncontr разброс относительно прямой и не видно особого тренда, как и у n_resident. Более плотно точки находятся около прямой для n_base_stations, если не учитывать выбросы.  

__Посмотрим на несколько графиков зависимости__ среднего числа звонков от признаков, с которыми слабая корреляция.

__Влияние минимального расстояния до объектов:__

In [ ]:
fig, ax_list = plt.subplots(2, 3, figsize=(16, 10))

ax_num = 0
for p in p_list:
    sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
                x='{}_min'.format(p), 
                y='calls_daily',
                ax=ax_list.reshape(-1, 1)[ax_num][0]);
    ax_num += 1

Признаки выглядят неинформативными, связи на графиках не видно. 

__Влияние среднего расстояния до объектов:__

In [ ]:
fig, ax_list = plt.subplots(2, 3, figsize=(16, 10))

ax_num = 0
for p in p_list:
    sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
                x='{}_mean_dist_k_5'.format(p), 
                y='calls_daily',
                ax=ax_list.reshape(-1, 1)[ax_num][0]);
    ax_num += 1

__Влияние числа объектов из full_house_info:__

In [ ]:
fig, ax_list = plt.subplots(2, 2, figsize=(16, 8))

sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
            x='n_old', 
            y='calls_daily',
            ax=ax_list[0][0]);
sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
            x='n_ind_proj', 
            y='calls_daily',
            ax=ax_list[0][1]);
sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
            x='n_auto_fire', 
            y='calls_daily',
            ax=ax_list[1][0]);
sns.regplot(data=pd.concat([train_features, calls_train], axis=1),
            x='n_without_gas', 
            y='calls_daily',
            ax=ax_list[1][1]);

Стоит как минимум оставить n_ind_proj. 

__Еще одно наблюдение__

Посмотрим на то, как выглядит график зависимости звонков от одного из "хороших" признаков, например, от числа магазинов в радиусе 250 м, если из звонков выбросить аномальные примеры:

In [ ]:
sns.regplot(data=pd.concat([train_features,
                            calls_train[calls_train.calls_daily < calls_train.calls_daily.quantile(0.995)]], axis=1),
                            x='shop_points_in_0.25', 
                            y='calls_daily');

И вот здесь становится грустно, потому что основная масса неаномальных точек просто кластеризуется в левом нижнем углу, а оставшиеся точки разбросаны относительно построенной прямой и никакого линейного тренда, да и в принципе тренда, не выделяется. 

Еще пример. Здесь вроде как можно увидеть тренд (по крайней мере, с ростом числа БС растет минимальное число звонков), но все еще с разбросом:

In [ ]:
sns.regplot(data=pd.concat([train_features,
                            calls_train[calls_train.calls_daily < calls_train.calls_daily.quantile(0.995)]], axis=1),
                            x='n_base_stations', 
                            y='calls_daily');

__Выводы по анализу:__

1) Видны выбросы в числе звонков, что может объясняться крупными, но редкими ЧП. Если использовать линейную модель, то их, думаю, не стоит брать.  

2) Про возможные причины корреляций и зависимостей — в промежуточных выводах выше. Субъективно выделяются менее информативные признаки, которые можно проверить на lasso-регрессии, если все-таки использовать в качестве модели регрессию, и отбросить, если их веса занулятся. Либо вообще не трогать при использовании случайного леса, например: возможно, из них можно вытащить что-то полезное, если по-честному смотреть на критерий информативности, а не просто глазами на матрицу корреляций и графики. 

3) Среди относительно информативных признаков нет чудо-признаков, которые бы хорошо в одиночку объясняли целевую переменную. В лучшем случае можно выделить что-то похожее на зависимось и проверить, как самые информативные признаки будут работать вместе. Это влияет на дальнейший ход задачи: если бы было больше времени, наверное, стоило бы потратить часть его на более серьезный data mining, чем у меня: например, погулять по соцсетям и поанализировать посты с геотегами. 

## Создание признаков II <a name="feat_2"></a>

[К началу](#cont)

При анализе целевого признака были видны выбросы: области с аномально большим числом звонков. __Введем бинарный признак__  "есть ли по соседству участок с большим числом звонков". "По соседству" пусть означает, что расстояние между центрами участков не больше 550 м (если рассматривать участки как идеальные квадраты ровно 500х500 м, то расстояние было бы 500 м, еще 50 м накинем сверху из-за возможной неточности). 

Мотивация: с одной стороны, стоит выкинуть аномальные участки, чтобы не портить модель, чувствительную к выбросам. С другой, хочется как-то сохранить часть информации с этих участков. И даже если нетипичные участки не выкидывать, этот признак, возможно, поможет отделить часть участков не с аномально высоким, но с повышенным числом звонков, основываясь на соседстве с этими нетипичными участками. 

In [ ]:
def many_calls_neighbour(zone_centre, many_calls_zones=many_calls_zones):
    '''
    in: array zone_centre (координаты центра участка lat_c, lon_c)
    out: 1, если участок рядом с участком с высоким числом звонков, 0 иначе
    '''
    for many_calls_zone in many_calls_zones[['lat_c', 'lon_c']].values:
        if distance(zone_centre, many_calls_zone) <= 0.55:
            return 1
    return 0    

In [ ]:
mcn = zones_df[(zones_df.is_target == 1) | (zones_df.is_test == 1)]. \
      apply(lambda row: many_calls_neighbour(np.array([row.lat_c, row.lon_c])), axis=1)

In [ ]:
train_features['mcn'] = mcn[idx_train]
test_features['mcn'] = mcn[idx_test]

## Выбор моделей и метрики <a name="model"></a>

[К началу](#cont)

Для оценки предсказаний используется тау Кендалла, и в задаче правильное упорядочивание важнее, чем точные численные ответы. Есть предположение о том, что похожие участки со схожими условиями проживания будут иметь похожее число звонков, а непохожесть наоборот позволит сравнивать число звонков в терминах меньше/больше — поэтому попробуем использовать метод ближайших соседей. Еще попробуем применить случайный лес из тех же соображений: лес сможет разбить объекты выборки на группы со схожими параметрами и схожим числом звонков, при этом обе эти модели смогут смоделировать нелинейную связь между признаками и числом звонков. А в конце посмотрим на то, как заработает градиентный бустинг. 

Метрика качества будет, понятно, тау Кендалла, раз она используется при оценке предсказаний.

Так как предсказания по дням недели оцениваются независимо, будем пока в качестве целевого признака брать среднее число звонков по всем дням (calls_daily). То есть получится, что для каждого участка предсказанное число звонков с понедельника по воскресенье будет одинаковым. 

## Предобработка данных и обучение <a name="learning"></a>

### KNeighborsRegressor <a name="knr"></a>

[К началу](#cont)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import validation_curve, learning_curve, KFold
from sklearn.pipeline import Pipeline

В признаках вида prt_* много NaN (больше половины), заменять их средним/медианным значением или 0 будет слишком неинформативно, поэтому их в качестве признаков брать не будем. Бинарные признаки, слабо коррелирующие с целевой переменной, тоже попробуем не использовать.

Метод ближайших соседей чувствителен к масштабированию, так что используем StandardScaler. Участки с аномально большим числом звонков тоже пока выбросим. От большой размерности признакового пространства KNN может пострадать, так что используем PCA.

__Построим кривую валидации и подберем параметр n_neighbors__

In [ ]:
def kendalltau_scorer(estimator, X, y):
    '''
    Возвращает коэффициент корреляции Кендалла;
    такой формат используется для совместимости с validation_curve и learning_curve
    '''
    from scipy.stats import kendalltau
    return kendalltau(estimator.predict(X), y).correlation

Подбирать будем только один гиперпараметр модели: число ближайших соседей, по которым путем усреднения числа звонков число звонков будет предсказываться для нового объекта. Подбирать будем на кросс-валидации (пятикратной, потому что данных не слишком много; с перемешиванием, раз временных меток у данных нет и сохранять временную структуру не надо; с помощью простого перебора). 

In [ ]:
%%time
drop_cols = ['prt_old', 'prt_ind_proj', 'prt_without_gas', 'prt_auto_fire',
             'prt_crossing', 'has_clinic', 'has_school', 'has_police']
knr_params = range(1, 61)
knr_pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=0.95, random_state=27)),
                     ('knr', KNeighborsRegressor())])

train_score, test_score = validation_curve(knr_pipe, 
                                           train_features.drop(drop_cols, axis=1).drop(many_calls_zones.index),
                                           calls_train['calls_daily'].drop(many_calls_zones.index),
                                           'knr__n_neighbors', knr_params,
                                           cv=KFold(n_splits=5, shuffle=True, random_state=27), 
                                           scoring=kendalltau_scorer)

In [ ]:
plt.plot(knr_params, train_score.mean(axis=1), label='train score')
plt.plot(knr_params, test_score.mean(axis=1), label='test score');
plt.xlabel('n_neighbors')
plt.xticks(np.arange(2, 61, 6))
plt.ylabel('kendall tau')
plt.legend();

В районе n_neighbors=8 у среднего скора на тесте максимум, кривая для теста близко к кривой для трейна, значит, можно предположить, что модель не переобучилась. Возьмем n_neighbors, отвечающий за максимальный средний тестовый скор.

In [ ]:
n_neighbors = knr_params[np.argmax(test_score.mean(axis=1))]
n_neighbors

И посмотрим на максимальный средний тестовый скор.

In [ ]:
test_score.mean(axis=1).max()

__Построим кривую обучения с выбранным n_neighbors__

In [ ]:
%%time
train_sizes=np.linspace(0.02, 1, 20)
knr_pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=0.95, random_state=27)),
                     ('knr', KNeighborsRegressor(n_neighbors=n_neighbors))])
N_train, train_score, test_score = learning_curve(knr_pipe, 
                                         train_features.drop(drop_cols, axis=1).drop(many_calls_zones.index),
                                         calls_train['calls_daily'].drop(many_calls_zones.index),
                                         train_sizes=train_sizes,
                                         cv=KFold(n_splits=5, shuffle=True, random_state=27), 
                                         scoring=kendalltau_scorer)

In [ ]:
plt.plot(N_train, train_score.mean(axis=1), label='train score')
plt.plot(N_train, test_score.mean(axis=1), label='test score');
plt.xlabel('train size')
plt.ylabel('kendall tau')
plt.legend();

Кривые для трейна и теста еще не сошлись, у обоих видна тенденция на рост с увеличением размера выборки. 

Попробуем __сделать предсказания этой модели на тесте и обучить еще одну__ на всех данных из обучающей выборки (до этого использовались только целевые участки). Признаков, соответственно, станет меньше, потому что некоторые получать долго и дорого и они считались только для целевых участков.  

In [ ]:
def write_submission_file(y_pred, filename, idx_test=idx_test):
    
    '''
    Записывает предсказанные результаты в файл;
    y_pred -- предсказания для числа звонков в среднем по всем дням
    '''
    from collections import OrderedDict
    
    calls = ['calls_wd{0}'.format(i) for i in range(0, 7)]
    pred_df = pd.DataFrame(OrderedDict([(col_name, y_pred) for col_name in calls]), index=idx_test)
    pred_df.to_csv(filename)

__Масштабирование, PCA__

В пайплайне выше масштабировался и бинарный признак mcn, потому что не хотелось с этим возиться, здесь же его масштабировать не будем. 

In [ ]:
scaler = StandardScaler()

drop_cols = ['prt_old', 'prt_ind_proj', 'prt_without_gas', 'prt_auto_fire',
             'prt_crossing', 'has_clinic', 'has_school', 'has_police', 'mcn']

scaler.fit(train_features.drop(drop_cols, axis=1).drop(many_calls_zones.index))
train_features_scaled = np.concatenate([scaler.transform(train_features.drop(drop_cols, axis=1).drop(many_calls_zones.index)), 
                                       train_features['mcn'].drop(many_calls_zones.index).values.reshape(-1, 1)], axis=1)
test_features_scaled = np.concatenate([scaler.transform(test_features.drop(drop_cols, axis=1)), 
                                       test_features['mcn'].values.reshape(-1, 1)], axis=1)    

pca = PCA(n_components=0.95, random_state=27)

pca.fit(train_features_scaled)
train_features_pca = pca.transform(train_features_scaled)
test_features_pca = pca.transform(test_features_scaled)

__Обучение с лучшим гиперпараметром, предсказание для теста__

In [ ]:
knr = KNeighborsRegressor(n_neighbors=n_neighbors)

knr.fit(train_features_pca, calls_train['calls_daily'].drop(many_calls_zones.index))
calls_pred = knr.predict(test_features_pca)
write_submission_file(calls_pred, 'knr.csv')

__Результат на LB__

34 место из 63 на тот момент.

<img src="https://habrastorage.org/webt/tx/ye/1o/txye1o9annxuglvpxoh-goa6u_i.png" />

### KNeighborsRegressor на расширенной выборке <a name="knr_ext"></a>

[К началу](#cont)

__Дополним два признака и создадим новые выборки__

In [ ]:
%%time
n_base_stations_non_target = zones_df[zones_df.is_target == 0] \
                            .apply(lambda row: obj_count((row.lat_bl, row.lon_bl, row.lat_tr, row.lon_tr),
                                                stat_coords), axis=1)

In [ ]:
n_base_stations_ext = pd.concat([n_base_stations, n_base_stations_non_target]).sort_index()

In [ ]:
idx_train_ext = zones_df[zones_df.is_test == 0].index
calls_train_ext = zones_df.loc[idx_train_ext, :].iloc[:, 8:]

quant_99 = calls_train_ext['calls_daily'].quantile(0.99)
mask = calls_train_ext['calls_daily'] > quant_99

many_calls_zones_ext = zones_df.loc[idx_train][mask]

In [ ]:
%%time
mcn_non_target = zones_df[zones_df.is_target == 0]. \
                 apply(lambda row: many_calls_neighbour(np.array([row.lat_c, row.lon_c]),
                                                        many_calls_zones=many_calls_zones_ext), axis=1)

In [ ]:
mcn_ext = pd.concat([mcn, mcn_non_target]).sort_index()

In [ ]:
full_features_ext = pd.concat([dist_features,
                               crossing_features,
                               n_base_stations_ext.rename('n_base_stations'),
                               mcn_ext.rename('mcn')],
                               axis=1)

In [ ]:
train_features_ext = full_features_ext.loc[idx_train_ext]
test_features_ext = full_features_ext.loc[idx_test]

__Посмотрим на кривую валидации__

In [ ]:
%%time
knr_params = range(1, 21)
knr_pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=0.95, random_state=27)),
                     ('knr', KNeighborsRegressor())])

train_score, test_score = validation_curve(knr_pipe, 
                                           train_features_ext.drop('prt_crossing', axis=1).drop(many_calls_zones_ext.index),
                                           calls_train_ext['calls_daily'].drop(many_calls_zones_ext.index),
                                           'knr__n_neighbors', knr_params,
                                           cv=KFold(n_splits=5, shuffle=True, random_state=27), 
                                           scoring=kendalltau_scorer)

In [ ]:
plt.plot(knr_params, train_score.mean(axis=1), label='train score')
plt.plot(knr_params, test_score.mean(axis=1), label='test score');
plt.xlabel('n_neighbors')
plt.xticks(np.arange(2, 21))
plt.ylabel('kendall tau')
plt.legend();

Кривые не сходятся, как в прошлый раз, но не сильно отличаются.

In [ ]:
n_neighbors = knr_params[np.argmax(test_score.mean(axis=1))]
n_neighbors

In [ ]:
test_score.mean(axis=1).max()

__Масштабирование, PCA__

In [ ]:
scaler = StandardScaler()

scaler.fit(train_features_ext.drop('prt_crossing', axis=1).drop(many_calls_zones_ext.index))
train_features_ext_scaled = np.concatenate([scaler.transform(train_features_ext.drop('prt_crossing', axis=1).drop(many_calls_zones_ext.index)), 
                            train_features_ext['mcn'].drop(many_calls_zones_ext.index).values.reshape(-1, 1)], axis=1)
test_features_ext_scaled = np.concatenate([scaler.transform(test_features_ext.drop('prt_crossing', axis=1)), 
                           test_features_ext['mcn'].values.reshape(-1, 1)], axis=1)    

pca = PCA(n_components=0.95, random_state=27)

pca.fit(train_features_ext_scaled)
train_features_ext_pca = pca.transform(train_features_ext_scaled)
test_features_ext_pca = pca.transform(test_features_ext_scaled)

__Обучение с лучшим гиперпараметром, предсказание для теста__

In [ ]:
knr = KNeighborsRegressor(n_neighbors=n_neighbors)

knr.fit(train_features_ext_pca, calls_train_ext['calls_daily'].drop(many_calls_zones_ext.index))
calls_pred = knr.predict(test_features_ext_pca)
write_submission_file(calls_pred, 'knr_extended.csv')

__Результаты на LB__

Все здорово просело. По всей видимости, учитывать нецелевые участки все-таки не стоит, либо стоит как-то по-хитрому, например, отсекая участки с совсем маленьким числом звонков и брать топ-n по звонкам. Если останется время, то можно попробовать и это. 

<img src="https://habrastorage.org/webt/id/6e/-u/id6e-urdmvodkr7_db8vxsfdjgq.png" />

### RandomForestRegressor <a name="rfr"></a>

[К началу](#cont)

Здесь строить кривые валидации и обучения уже не будем, просто посмотрим на cross_val_score, параметры у валидации те же. Гиперпараметры для леса тоже особо подбирать не будем и воспользуемся советами из [статьи курса](https://habrahabr.ru/company/ods/blog/324402/#2-sluchaynyy-les) насчет размера подмножества признаков и минимального числа объектов в листах. Деревьев попробуем взять 500 (лес не переобучается от количества деревьев, так что можно и больше).

От выбросов лес не пострадает, так что и участки с аномальным числом звонков выбрасывать не будем.

In [ ]:
drop_cols = ['prt_old', 'prt_ind_proj', 'prt_without_gas', 'prt_auto_fire', 'prt_crossing']

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

n_features = train_features.drop(drop_cols, axis=1).shape[1]
rfr = RandomForestRegressor(n_estimators=500, min_samples_leaf=5, max_features=round(n_features / 3), random_state=27)
cvs = cross_val_score(estimator=rfr, X=train_features.drop(drop_cols, axis=1), y=calls_train['calls_daily'],
                      scoring=kendalltau_scorer, cv=KFold(n_splits=5, shuffle=True, random_state=27))

In [ ]:
cvs

In [ ]:
cvs.mean(), cvs.std()

In [ ]:
rfr.fit(train_features.drop(drop_cols, axis=1), calls_train['calls_daily'])

__Посмотрим на значимость признаков__

Некоторые новые признаки, которые создавались вручную не на основе StarterKit, оказались в топ-20: число зон-соседей с большим числом звонков, число жилых домов в участке, число БС в участке. Еще для леса оказались важными признаки вроде минимальное/максимальное расстояние до какого-то объекта, хотя предполагалось, что эти признаки заработают хуже, чем число каких-то объектов в участке. 

In [ ]:
importances = rfr.feature_importances_
indices = np.argsort(importances)[::-1]

sns.barplot(y=train_features.drop(drop_cols, axis=1).columns[indices[:21]],
            x=importances[indices[:21]],
            orient='h');

In [ ]:
calls_pred = rfr.predict(test_features.drop(drop_cols, axis=1))
write_submission_file(calls_pred, 'rfr.csv')

__Результаты на LB__

14 место из 65 на тот момент.

<img src="https://habrastorage.org/webt/jk/-h/8u/jk-h8uembrj7xszlnhswynsjdbk.png" />

### XGBRegressor <a name="xgb"></a>

[К началу](#cont)

In [ ]:
# !pip install xgboost

In [ ]:
from xgboost.sklearn import XGBRegressor

В качестве базовых алгоритмов возьмем деревья. Настраивать гиперпараметры будем в два этапа: сначала для базовых алгоритмов (max_depth, максимальную глубину деревьев; colsample_bytree, долю признаков, на которой будет обучаться каждое дерево, и gamma,  минимальное уменьшение функции потерь, при котором вершина дерева еще будет разбиваться). 

max_depth и gamma напрямую контролируют сложность базовой модели, а colsample_bytree добавляет шума для большей устойчивости модели ([документация XGBoost](http://xgboost.readthedocs.io/en/latest/how_to/param_tuning.html?highlight=parameters)). Интервалы перебора max_depth и colsample_bytree берутся навскидку, gamma с предварительной оценкой.

Потом настроим темп обучения и число деревьев в модели. Также оставим отложенную выборку и результаты обоих этапов настройки гиперпараметров оценим на ней.

__Оценим возможный интервал для gamma__

__Подберем первый блок параметров__

Решается задача регрессии, в качестве функции потерь будет по умолчанию использоваться RMSE. 

$$RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{({\hat{y_i} - y_i})^2}}$$ 

Поскольку предсказываются доли, то максимальной RMSE будет, когда всегда предсказывается число звонков, равное единице, когда на самом деле звонков всегда ноль. Тогда RMSE лежит в интервале от 0 до 1.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_features.drop(drop_cols, axis=1),
                                                      calls_train['calls_daily'],
                                                      test_size=0.2,
                                                      random_state=27)

xgbr_params = {'max_depth': np.arange(3, 16, 3),
               'colsample_bytree': [0.3, 0.6, 0.9], 
               'gamma': np.append(np.logspace(-4, -1, 4), 0)}

cv = GridSearchCV(estimator=XGBRegressor(random_state=27), param_grid=xgbr_params, scoring=kendalltau_scorer,
                  cv=KFold(n_splits=5, shuffle=True, random_state=27), n_jobs=-1)

In [ ]:
%%time
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_, cv.best_score_

__Оценим на отложенной выборке__

In [ ]:
kendalltau(cv.predict(X_valid), y_valid).correlation

__Посмотрим на второй блок параметров__

Попробуем увеличить число деревьев и снизить темп обучения (по умолчанию 0.1), learning_rate, при этом увеличив число итераций, num_boost_round (по умолчанию 10). Будем использовать оптимальные параметры с кросс-валидации выше.

In [ ]:
xgbr_params = {'n_estimators': [200, 300, 400, 500],
               'learning_rate': [0.01, 0.001]}
set_params = {'max_depth': 15, 
              'gamma': 0,
              'colsample_bytree': 0.6,
              'random_state': 27,
              'num_boost_round': 20}

cv = GridSearchCV(estimator=XGBRegressor(**set_params),
                  param_grid=xgbr_params, scoring=kendalltau_scorer,
                  cv=KFold(n_splits=5, shuffle=True, random_state=27), n_jobs=-1)

In [ ]:
%%time
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_, cv.best_score_

__И оценим на отложенной выборке__

In [ ]:
kendalltau(cv.predict(X_valid), y_valid).correlation

__Обучим xgb на всей выборке и сделаем предсказания на тестовой__

In [ ]:
final_params = {'max_depth': 15, 
                'gamma': 0,
                'colsample_bytree': 0.6,
                'random_state': 27,
                'num_boost_round': 20,
                'learning_rate': 0.01,
                'n_estimators': 500}
xgbr = XGBRegressor(**final_params)
xgbr.fit(train_features.drop(drop_cols, axis=1), calls_train['calls_daily'])

In [ ]:
write_submission_file(xgbr.predict(test_features.drop(drop_cols, axis=1)), 'xgbr.csv')

__Результат на LB__

Хуже, чем у леса. Вероятно, стоит еще повозиться с гиперпараметрами, например, подбирать число итераций и параметры ранней остановки тоже на кросс-валидации. 

<img src="https://habrastorage.org/webt/kz/cz/af/kzczafhzo7irgs2rmc7_9lt4a_i.png" />

## Почти выводы <a name="alm_conc"></a>

[К началу](#cont)

Наткнулась на [публичный кернел](http://nbviewer.jupyter.org/github/VlasovKirill/mts-geohack/blob/master/2-public_ranking.ipynb) от Кирилла Власова и воспользовалась его идеей преобразовать целевой признак так, чтобы его распределение было менее ассимметричное и больше похоже на нормальное, что, в предположении, должно лучше сохранять порядок зон по числу звонков.

Преобразование — корень от логарифма (log1p) целевого признака. На графики, где видно, как изменяется целевой признак, можно посмотреть в кернеле. 

In [ ]:
%%time

n_features = train_features.drop(drop_cols, axis=1).shape[1]
rfr = RandomForestRegressor(n_estimators=500, min_samples_leaf=5, max_features=round(n_features / 3), random_state=27)
cvs = cross_val_score(estimator=rfr, X=train_features.drop(drop_cols, axis=1), y=np.sqrt(np.log1p(calls_train['calls_daily'])),
                      scoring=kendalltau_scorer, cv=KFold(n_splits=5, shuffle=True, random_state=27))

In [ ]:
cvs, cvs.mean(), cvs.std()

In [ ]:
rfr.fit(train_features.drop(drop_cols, axis=1), np.sqrt(np.log1p(calls_train['calls_daily'])))
calls_pred = rfr.predict(test_features.drop(drop_cols, axis=1))
calls_pred = np.expm1(calls_pred ** 2)
write_submission_file(calls_pred, 'rfr_calls_modelled.csv')

__Результаты на LB__

С лесом и преобразованием получилось 10 место из 71 на тот момент.

<img src="https://habrastorage.org/webt/fv/pc/as/fvpcasemde11qmwshqq9rddp1ly.png" />

__Текущий рейтинг__

Посылок больше, чем тех, которые выносились в этот проект: в проекте пропускались первые пробные посылки на основе стартера.

<img src="https://habrastorage.org/webt/k2/nt/rk/k2ntrklewoe82f9xhc8hlkkzme4.png" />

## Выводы <a name="conc"></a>

[К началу](#cont)

Если по-честному, то корреляция в 67% выглядит довольно маленькой и ненадежной, если решение всерьез применять на практике. Наверное, отчасти это объясняется сложностью моделируемого процесса, в котором много случайности, хотя есть и какие-то закономерности, которые модель улавливает. Думаю, чтобы решение действительно заработало, нужно побольше данных в обучающей выборке и больше информативных признаков. С признаками вообще получается одновременно и сложнее, и проще, чем в соревнованиях, где внешние данные не разрешены: проще, потому что можно брать вообще все, что одобрят организаторы, и сложнее, потому что нужно много времени тратить на обдумывание того, что нужно искать, сам поиск и предобработку. Зато интересно. 

Много чего можно улучшить в самом решении, конечно:

1) Использовать данные по ЧП (статистику пожаров и ДТП, например). Проблема: сходу такие данные в удобном формате не ищутся, много возни с обработкой. Плюс такие данные полезны для соревнования, но для реального применения на практике — не очень, потому что признаки на их основе были бы по сути построены на статистике из будущего.

2) Использовать данные из соцсетей и, возможно, часть нецелевых участков, об этом писала выше.

3) Использовать данные, которые хотелось использовать, если бы они нашлись: хотела построить признаки на основе данных по жилым домам с камерами видеонаблюдения во дворе/в подъезде, но нашла такие данные только для Москвы, но не для МО.

4) Использовать более сложные модели; получше настроить xgboost, который должен зайти в этой задаче, потому что признаков немного; либо вообще подойти к задаче не как к задаче регрессии ([вот здесь](http://nbviewer.jupyter.org/github/VlasovKirill/mts-geohack/blob/master/2-public_ranking.ipynb) Кирилл Власов подходит к задаче как к задаче ранжирования: соответственно, меняется функция потерь и можно более явно оптимизировать именно порядок, а не точные значения; я о таком подходе не думала.

5) Попробовать решать задачу как классификацию: кластеризовать участки из трейна по какому-то подмножеству информативных признаков, таким образом получить кластеры-классы.

6) Поработать с целевым признаком: например, отдельно предсказывать разные дни недели, учитывать рабочие/нерабочие дни.
